In [1]:
import sys
 
# setting path
sys.path.append('../')
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from citylearn.citylearn import CityLearnEnv
from citylearn_wrapper import CityLearnEnvWrapper
import pandas as pd
import numpy as np
import csv
from multiprocessing.spawn import prepare
from turtle import forward
from typing import Any

from planning_model import LitPlanningModel

In [2]:
env_config = {
    "schema": "../data/citylearn_challenge_2022_phase_1/schema.json",
}
hidden_size = 256
batch_norm = True
num_episodes = 50

In [3]:
env = CityLearnEnvWrapper(env_config)
obs_size = env.observation_space.shape[0]
act_size = env.action_space.shape[0]


In [4]:
class PlanningDataset(Dataset):
    def __init__(self, filename) -> None:
        super().__init__()
        self.filename = filename
        data = pd.read_csv(filename, delimiter="|").dropna()
        self.X = np.array([eval(data.iloc[i, 0]) + eval(data.iloc[i, 2]) for i in range(len(data))])
        self.X_mean = np.mean(self.X, axis=0)
        self.X_std = np.std(self.X, axis=0)
        self.y = np.array([eval(data.iloc[i, 1]) for i in range(len(data))])
        self.y_mean = np.mean(self.y, axis=0)
        self.y_std = np.std(self.y, axis=0)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return torch.tensor(self.X[index], dtype=torch.float32), torch.tensor(self.y[index], dtype=torch.float32)



In [5]:
dataset = PlanningDataset("planning_model_data.csv")

train_size = int(len(dataset) * 0.8)
val_size = len(dataset) - train_size

#train_set, val_set = torch.utils.data.random_split(dataset, [train_size, val_size])
val_base_idxs = np.array(list(range(8760//5)))
train_base_idxs = np.array(list(range(8760//5, 8760)))
val_idxs = [val_base_idxs + i * 8760 for i in range(len(dataset) // 8760)]
train_idxs = [train_base_idxs + i * 8760 for i in range(len(dataset) // 8760)]
val_idxs = np.concatenate(val_idxs)
train_idxs = np.concatenate(train_idxs)
train_set = torch.utils.data.Subset(dataset, train_idxs)
val_set = torch.utils.data.Subset(dataset, val_idxs)
train_loader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=3)
val_loader = DataLoader(val_set, batch_size=128, shuffle=False, num_workers=3)

In [6]:
model = LitPlanningModel(obs_size, act_size, hidden_size, X_mean=dataset.X_mean, y_mean=dataset.y_mean, X_std=dataset.X_std, y_std=dataset.y_std)

In [7]:
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint

wandb_logger = WandbLogger(project="active-rl-planning-model", entity="social-game-rl", log_model="all")
wandb_logger.experiment.config["exp_name"] = "alternate_validation_set"
checkpoint_callback = ModelCheckpoint(monitor="val_loss")
trainer = pl.Trainer(gpus=1, precision=32, logger=wandb_logger, callbacks=[checkpoint_callback], auto_lr_find=False, max_epochs=100)

#trainer.tune(model, train_dataloaders = train_loader, val_dataloaders = val_loader)
trainer.fit(model, train_dataloaders = train_loader, val_dataloaders = val_loader)

/home/doseok/miniconda3/envs/ActiveRL/lib/python3.10/site-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Currently logged in as: doseok (social-game-rl). Use `wandb login --relogin` to force relogin
/home/doseok/miniconda3/envs/ActiveRL/lib/python3.10/site-packages/wandb/sdk/lib/ipython.py:47: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML  # type: ignore


/home/doseok/miniconda3/envs/ActiveRL/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type       | Params
--------------------------------------
0 | layers | ModuleList | 231 K 
--------------------------------------
231 K     Trainable params
0         Non-trainable params
231 K     Total params
0.927     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/doseok/miniconda3/envs/ActiveRL/lib/python3.10/site-packages/pytorch_lightning/core/module.py:555: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  value = torch.tensor(value, device=self.device)


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
